In [1]:
!git clone https://github.com/Ayan-Ansar/Eng2HingTranslator.git

Cloning into 'Eng2HingTranslator'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 26 (delta 6), reused 11 (delta 0), pack-reused 0
Receiving objects: 100% (26/26), 17.89 KiB | 345.00 KiB/s, done.
Resolving deltas: 100% (6/6), done.
Filtering content: 100% (2/2), 686.65 MiB | 30.47 MiB/s, done.


In [2]:
%cd '/content/Eng2HingTranslator'

/content/Eng2HingTranslator


In [3]:
!pip install datasets transformers[sentencepiece] -q
!pip install indic-transliteration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 2.9 MB/s eta 0:00:00


In [4]:
model_checkpoint = "t5-small"
model_path = '/content/Eng2HingTranslator/saved_models/'

In [20]:
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM
import numpy as np
import nltk
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
nltk.download('words')
from nltk.corpus import words
import warnings

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [21]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [7]:
# load the tf-small tokenizer and the pretrained model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_path)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/Eng2HingTranslator/saved_models/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [68]:
common_hindi_eng_words = ['to','agar','me','hoon','ye','ka','mein']

In [69]:
def translate_to_hindi(word):
    try:
        translation = transliterate(word, sanscript.ITRANS, sanscript.DEVANAGARI)
        return translation
    except Exception as e:
        return word

def translate(input_text):
    tokenized = tokenizer([input_text], return_tensors='np')
    out = model.generate(**tokenized, max_length=128)
    with tokenizer.as_target_tokenizer():
        result = tokenizer.decode(out[0], skip_special_tokens=True)
    print('Translated:', result)
    result = result.lower().split(' ')
    ans = []
    for i in result:
        if i in common_hindi_eng_words:
            i = translate_to_hindi(i)
        if i not in words.words():
            i = translate_to_hindi(i)
        ans.append(i)

    return ' '.join(ans)

In [70]:
INPUT = ["Definitely share your feedback in the comment section",
         "So even if it's a big video, I will clearly mention all the products",
         "I was waiting for my bag"]

In [71]:
for i, text in enumerate(INPUT):
    print(f"Input Text {i+1}: ", text)
    output_text = translate(text)
    print(f"Converted to mix enlgish-hindi Output Text {i+1}: ", output_text)
    print('\n')

Input Text 1:  Definitely share your feedback in the comment section
Translated: Definitely apke comment section me share kare
Converted to mix enlgish-hindi Output Text 1:  definitely अप्के comment section मे share करे


Input Text 2:  So even if it's a big video, I will clearly mention all the products
Translated: To agar ye bada video hai to main sabhi products ko mention karunga
Converted to mix enlgish-hindi Output Text 2:  तो अगर् ये बद video है तो main सभि प्रोदुच्त्स् ko mention करुन्ग


Input Text 3:  I was waiting for my bag
Translated: mai apna bag ka wait kar raha hoon
Converted to mix enlgish-hindi Output Text 3:  मै अप्न bag क wait कर् रह हून्


